In [ ]:
import requests
from collections import Counter
from google.colab import drive
!pip install pyngrok
from pyngrok import ngrok
from flask import Flask, jsonify
!ngrok authtoken 2lCRac4m9bkOWWsRMSao6A4c5yU_txPeh33rRQhfktTiUoXN       # 여기에 ngrok authtoken을 넣으세요

# 1. Edamam API 설정
APP_ID = "9a589b73"  # 여기 YOUR_APP_ID에 발급받은 Application ID를 넣으세요
APP_KEY = "fd1e7ac58907520461692ba37b65e191"  # 여기 YOUR_APP_KEY에 발급받은 Application Key를 넣으세요

# 2. 텍스트 파일에서 재료를 읽어오는 함수
def load_ingredients_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.read().split()  # 파일에서 모든 단어를 공백을 기준으로 분리

    # Counter를 사용해 각 재료의 반복 횟수를 계산
    ingredients = Counter(lines)
    return dict(ingredients)

# 3. 레시피 검색 함수
def get_recipes(ingredients):
    url = "https://api.edamam.com/search"

    # 파라미터 설정
    params = {
        'q': ','.join(ingredients.keys()),  # 검색할 재료들을 콤마로 구분하여 전달
        'app_id': APP_ID,
        'app_key': APP_KEY,
        'from': 0,  # 결과의 시작 인덱스
        'to': 10,  # 최대 10개의 결과를 반환
    }

    # GET 요청 보내기
    response = requests.get(url, params=params)

    # 응답 데이터 확인
    if response.status_code == 200:
        return response.json()  # JSON 형식으로 응답 반환
    else:
        print(f"Error: {response.status_code}")
        return None

# 4. 추가 재료 최소화 필터링 함수
def find_best_recipe(recipes, user_ingredients):
    best_recipe = None
    min_additional_ingredients = float('inf')
    additional_ingredients = set()

    for hit in recipes['hits']:
        recipe = hit['recipe']
        recipe_ingredients = {ingredient.lower() for ingredient in recipe['ingredientLines']}  # 레시피 재료들

        # 입력한 재료로 커버할 수 없는 재료 계산
        additional_ingredients = recipe_ingredients - set(user_ingredients.keys())
        if len(additional_ingredients) < min_additional_ingredients:
            min_additional_ingredients = len(additional_ingredients)
            best_recipe = recipe

    return best_recipe, min_additional_ingredients

# 5. 레시피 출력 함수
def print_best_recipe(recipe, additional_ingredients):
    if recipe:
        print(f"Best recipe found: {recipe['label']}")
        print(f"Ingredients: {', '.join(recipe['ingredientLines'])}")
        print(f"Additional ingredients needed: {', '.join(additional_ingredients)}")
        print(f"Cooking Instructions: {recipe['url']}\n")
    else:
        print("No suitable recipe found.")

# 6. 메인 함수
    # 1. Google Drive 마운트
drive.mount('/content/drive')

    # 2. 텍스트 파일이 Google Drive에 있다고 가정
    # 예시 경로: '/content/drive/My Drive/ingredients.txt'
file_path = '/content/drive/My Drive/ingredients.txt'  # 실제 파일 경로로 수정

    # 텍스트 파일에서 재료 로드
user_ingredients = load_ingredients_from_file(file_path)
print(f"Loaded ingredients: {user_ingredients}")

    # API를 통해 레시피 검색
recipes = get_recipes(user_ingredients)

if recipes:
        # 추가 재료 최소화된 레시피 찾기
    best_recipe, additional_ingredients_count = find_best_recipe(recipes, user_ingredients)
    additional_ingredients = list({ingredient.lower() for ingredient in best_recipe['ingredientLines']} - set(user_ingredients.keys()))

        # 결과 출력
    print_best_recipe(best_recipe, additional_ingredients)
else:
    print("No recipes found.")
    # ngrok 인증 토큰 설정
ngrok.set_auth_token("2lCRac4m9bkOWWsRMSao6A4c5yU_txPeh33rRQhfktTiUoXN")  # 여기에 ngrok authtoken을 넣으세요

# Flask 앱 설정
app = Flask(__name__)

@app.route('/', methods=['GET'])
def get_data():
  food_items = []
  for name, number in user_ingredients.items():
      item = {
          "name": name.capitalize(),  # 첫 글자를 대문자로 변환
          "number": str(number),       # 숫자를 문자열로 변환
      }
      food_items.append(item)

  recipe_data = {
      "label": best_recipe["label"],
      "ingredients": best_recipe["ingredientLines"],
      "additional_ingredients": additional_ingredients,
      "url": best_recipe["url"]
  }

  data = {
        "food_items": food_items,
        "recipe": recipe_data
  }
  return jsonify(data)

# ngrok을 통해 터널 연결
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

# Flask 앱 실행
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded ingredients: {'onion': 1, 'carrot': 1, 'egg': 1}
Best recipe found: Carrot Potato Latkes
Ingredients: 3/4 pound carrots (4 medium), 1 1/2 pounds russet (baking) potatoes (about 3), 1 large onion, 2 tablespoon fresh lemon juice, 2/3 cup all purpose flour, 3 large eggs, lightly beaten, 1 cup vegetable oil
Additional ingredients needed: 3 large eggs, lightly beaten, 1 cup vegetable oil, 2 tablespoon fresh lemon juice, 1 1/2 pounds russet (baking) potatoes (about 3), 3/4 pound carrots (4 medium), 1 large onion, 2/3 cup all purpose flour
Cooking Instructions: https://www.epicurious.com/recipes/food/views/carrot-potato-latkes-365212

 * ngrok tunnel "NgrokTunnel: "https://c29d-34-82-25-51.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
